In [ ]:
'''
政大中文學報
http://ctma.nccu.edu.tw/chibulletin/app/paper.php
'''

'''
匯入套件
'''
# 請求套件
import requests as req

# 格式化輸出工具
from pprint import pprint as pp

# HTML Parser
from bs4 import BeautifulSoup as bs

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# regular expression 工具
import re

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 建立隨機數
from random import randint

# 資料庫 (sqlite3)
import sqlite3

# Excel 工具
from openpyxl import load_workbook
from openpyxl import Workbook

# 時間工具
from datetime import datetime

# 其它
import json, os, sys

'''
設定
'''
# 主要首頁
prefix = "http://ctma.nccu.edu.tw/"
infix = 'chibulletin/app'
url = prefix + infix + '/paper.php'

# JSON 存檔路徑
json_path = "./政大中文學報.json"

# 自訂標頭
my_headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'
}

# 整理資料用的變數
listData = []

# 建立儲存檔案用的資料夾，不存在就新增
folderPath = '政大中文學報'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

'''
程式區域
'''
# 取得分頁資訊
def getPaginationLinks():
    # 建立 parser
    res = req.get(url, headers = my_headers)
    res.encoding = 'utf-8'
    soup = bs(res.text, "lxml")
    
    # 取得分頁資訊
    regex = r"\s(\d+)\s" 
    a_elms = soup.select('ol.PageList li a')
    for a in a_elms:
        match = re.search(regex, a.get_text())
        if match != None:
            listData.append({
                "page": match[1],
                "link": url + '' + a['href']
            })    

# 取得主要頁面所有連結資訊
def getMainLinks():
    for index, _dict in enumerate(listData):
        # 沒有 sub 屬性，則建立，為了放置細節頁的內容
        if "sub" not in listData[index]:
            listData[index]['sub'] = []
        
        # 建立 parser
        res = req.get(_dict['link'], headers = my_headers)
        res.encoding = 'utf-8'
        soup = bs(res.text, "lxml")
        
        # 取得主要連結
        a_elms = soup.select('table#RSS_Table_page_paper_1 tbody tr td a')
        for a in a_elms:
            listData[index]['sub'].append({
                "title": a.get_text(),
                "link": prefix + '/' + infix + '/' + a['href']
            })
    
# 取得 PDF 連結
def getPdfLinks():
    for index, _dict in enumerate(listData):
        for idx, _d in enumerate(listData[index]['sub']):
            # 沒有 sub 屬性，則建立，為了放置細節頁的內容
            if "downloads" not in listData[index]['sub'][idx]:
                listData[index]['sub'][idx]['downloads'] = []
                
            # 過濾出 pdf 連結
            regex = r"https?:\/\/.+\.pdf"
            
            # 建立 parser
            res = req.get(_d['link'], headers = my_headers)
            res.encoding = 'utf-8'
            soup = bs(res.text, "lxml")

            # 取得該頁的每一個 tr
            tr_elms = soup.select('table#RSS_Table_page_paper_1 tbody tr')
            if len(tr_elms) > 0:
                for tr in tr_elms:
                    # 取得該 tr 下的所有 td
                    td_elms = tr.select('td')
                    
                    # 期數
                    title = td_elms[0].get_text()
                    
                    # 作者
                    author = td_elms[1].get_text()
                    
                    # 頁碼
                    page = td_elms[2].get_text()
                    
                    # 篇名
                    journal_name = td_elms[3].get_text()
                    
                    # 篇名超連結
                    link = prefix + '/' + infix + '/' + td_elms[3].select_one('a')['href']
                    
                    # pdf 連結
                    if len(td_elms[4].select('a')) > 0:
                        pdf_link = prefix + td_elms[4].select('a')[1]['href']
                    else:
                        pdf_link = None

                    # 整理資料
                    listData[index]['sub'][idx]['downloads'].append({
                        "title": title,
                        "author": author,
                        "page": page,
                        "journal_name": journal_name,
                        "link": link,
                        "pdf_link": pdf_link
                    })
                    
                    # (選項) 下載 PDF
                    os.system(f"curl {pdf_link} -o {folderPath}/{journal_name}.pdf")

# 儲存成 JSON
def saveToJson():
    with open(f"{folderPath}/{json_path}", "w", encoding="utf-8") as file:
        file.write( json.dumps(listData, ensure_ascii=False, indent=4) )
        
'''
執行區域
'''
if __name__ == "__main__":
    getPaginationLinks()
    getMainLinks()
    getPdfLinks()
    saveToJson()